In [1]:
import azureml.core

print("Azure Machine Learning SDK Version:", azureml.core.VERSION)

Azure Machine Learning SDK Version: 1.37.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

ml-workspace | eastus | ml-rg


In [3]:
from azureml.core.experiment import Experiment

from files import BCTrade_utils
env ='Bitstamp_1'
experiment_name = BCTrade_utils.registerBCTrade(env)
exp = Experiment(workspace=ws, name=experiment_name)

In [4]:
from azureml.core import Run

#run_id = child_run_0.id # Or set to run id of a completed run (e.g. 'rl-cartpole-v0_1587572312_06e04ace_head')
run_id = 'BCTrade-Bitstamp_1_1645290884_751d5f6b_head'
child_run_0 = Run(exp, run_id=run_id)
#child_run_0.get_environment()

In [5]:
from os import path
from distutils import dir_util

training_artifacts_path = path.join("logs", "PPO")
print("Training artifacts path:", training_artifacts_path)

if path.exists(training_artifacts_path):
    dir_util.remove_tree(training_artifacts_path)

# Download run artifacts to local compute
child_run_0.download_files(training_artifacts_path)

Training artifacts path: logs/PPO


In [6]:
# A helper function to find checkpoint files in a directory
def find_checkpoints(file_path):
    print("Looking in path:", file_path)
    checkpoints = []
    for root, _, files in os.walk(file_path):
        for name in files:
            if os.path.basename(root).startswith('checkpoint_'):
                checkpoints.append(path.join(root, name))
    return checkpoints

checkpoint_files = find_checkpoints(training_artifacts_path)

Looking in path: logs/PPO


In [7]:
# Find checkpoints and last checkpoint number
checkpoint_numbers = []
for file in checkpoint_files:
    file = os.path.basename(file)
    if file.startswith('checkpoint-') and not file.endswith('.tune_metadata'):
        checkpoint_numbers.append(int(file.split('-')[-1]))

print("Checkpoints:", checkpoint_numbers)

last_checkpoint_number = max(checkpoint_numbers)
print("Last checkpoint number:", last_checkpoint_number)

Checkpoints: [1, 10, 2, 3, 4, 5, 6, 7, 718, 8, 9]
Last checkpoint number: 718


In [8]:
# Upload the checkpoint files and create a DataSet
from azureml.core import Dataset

datastore = ws.get_default_datastore()
checkpoint_dataref = datastore.upload_files(checkpoint_files, target_path='checkpoints_' + run_id, overwrite=True)
checkpoint_ds = Dataset.File.from_files(checkpoint_dataref)
print(checkpoint_ds)
print(checkpoint_ds.as_mount())
#inputs=[
#        checkpoint_ds.as_named_input('artifacts_dataset'),
#        checkpoint_ds.as_named_input('artifacts_path').as_mount()]

Uploading an estimated of 22 files
Uploading logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_1/checkpoint-1
Uploaded logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_1/checkpoint-1, 1 files out of an estimated total of 22
Uploading logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_1/checkpoint-1.tune_metadata
Uploaded logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_1/checkpoint-1.tune_metadata, 2 files out of an estimated total of 22
Uploading logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_10/checkpoint-10.tune_metadata
Uploaded logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_10/checkpoint-10.tune_metadata, 3 files out of an estimated total of 22
Uploading logs/PPO/PPO_BCTrade-Bitstamp_1_884635fc_2022-02-19_17-15-27psedcdsl/checkpoint_2/checkpoint-2.tune_metadata
Uploaded logs/PPO/PPO_BCTrade-Bitsta

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.
